# Fedbiomed Researcher base example with OPACUS

Use for developing (autoreloads changes made across packages)

In this notebook we show how `opacus` (https://opacus.ai/) can be used in Fed-BioMed. Opacus is a library which allows to train PyTorch models with differential privacy. We will train the basic MNIST example using two nodes.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from fedbiomed.researcher.requests import Requests
req = Requests()
req.list(verbose=True)

2022-01-20 13:02:36,486 fedbiomed INFO - Component environment:
2022-01-20 13:02:36,488 fedbiomed INFO - - type = ComponentType.RESEARCHER
2022-01-20 13:02:37,420 fedbiomed INFO - Messaging researcher_1eef641b-73f6-45ab-b859-e37b8d87bbfd successfully connected to the message broker, object = <fedbiomed.common.messaging.Messaging object at 0x10d2119a0>
2022-01-20 13:02:37,498 fedbiomed INFO - Listing available datasets in all nodes... 
2022-01-20 13:02:37,516 fedbiomed INFO - log from: node_063a33bf-26a8-436a-876e-ac5a0827c4b2 / DEBUG - Message received: {'researcher_id': 'researcher_1eef641b-73f6-45ab-b859-e37b8d87bbfd', 'command': 'list'}
2022-01-20 13:02:37,518 fedbiomed INFO - log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / DEBUG - Message received: {'researcher_id': 'researcher_1eef641b-73f6-45ab-b859-e37b8d87bbfd', 'command': 'list'}
2022-01-20 13:02:47,516 fedbiomed INFO - 
 Node: node_963c86ad-835e-424d-9bf5-73d8ec418720 | Number of Datasets: 1 
+--------+-------------+---

{'node_963c86ad-835e-424d-9bf5-73d8ec418720': [{'name': 'MNIST',
   'data_type': 'default',
   'tags': ['#MNIST', '#dataset'],
   'description': '50.0 percent of MNIST database',
   'shape': [30000, 1, 28, 28]}],
 'node_063a33bf-26a8-436a-876e-ac5a0827c4b2': [{'name': 'MNIST',
   'data_type': 'default',
   'tags': ['#MNIST', '#dataset'],
   'description': '50.0 percent of MNIST database',
   'shape': [30000, 1, 28, 28]}]}

## Start the network
Before running this notebook, start the network with `./scripts/fedbiomed_run network`

## Setting the node up
It is necessary to previously configure a node:
1. `./scripts/fedbiomed_run node add`
  * Select option 2 (default), and write MNIST to add MNIST to the node through `torchvision.datasets.CIFAR10`
  * Confirm default tags by hitting "y" and ENTER
  * Pick the folder where MNIST is downloaded (this is due torch issue https://github.com/pytorch/vision/issues/3549)
  * Data must have been added (if you get a warning saying that data must be unique is because it's been already added)
  
2. Check that your data has been added by executing `./scripts/fedbiomed_run node list`
3. Run the node using `./scripts/fedbiomed_run node run`. Wait until you get `Starting task manager`. it means you are online.

## Define an experiment model and parameters

Declare a torch.nn MyTrainingPlan class to send for training on the node

In [3]:
from fedbiomed.researcher.environ import environ
import tempfile
tmp_dir_model = tempfile.TemporaryDirectory(dir=environ['TMP_DIR']+'/')
model_file = tmp_dir_model.name + '/class_export_mnist.py'

In the cell below, we are going to define the model using opacus for differential privacy. For this example, we are going to use the function `make_private` from `opacus.privacy_engine`. Two hyperparameters should be defined:
* `noise_multiplier`: The ratio of the standard deviation of the Gaussian noise to the L2-sensitivity of the function to which the noise is added (How much noise to add)
* `max_grad_norm`: The maximum norm of the per-sample gradients. Any gradient with norm higher than this will be clipped to this value.

It is worth noting that in order to use the opacus `PrivacyEngine` class we need to properly define as training plan attributes a `model`, a `dataloader` and an `optimizer`.

In [4]:
%%writefile "$model_file"

import torch
import torch.nn as nn
from fedbiomed.common.torchnnDP import TorchTrainingDPPlan
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Here we define the model to be used. 
# You can use any class name (here 'Net')
class MyTrainingPlan(TorchTrainingDPPlan):
    def __init__(self, model_args):
        super(MyTrainingPlan, self).__init__()
        
        # Here we define the custom dependencies that will be needed by our custom Dataloader
        # In this case, we need the torch DataLoader classes
        # Since we will train on MNIST, we need datasets and transform from torchvision
        deps = ["from torchvision import datasets, transforms",
                "from torch.utils.data import DataLoader",]
        self.add_dependency(deps)
        
        self.diff_privacy = model_args['diff_privacy']
        self.privacy_func = model_args['privacy_func']
        self.noise_multiplier = model_args['noise_multiplier']
        self.max_grad_norm = model_args['max_grad_norm']
        
        self.model = self.make_model()
        
    def make_model(self):
        model = nn.Sequential(nn.Conv2d(1, 32, 3, 1),
                              nn.ReLU(),
                              nn.Conv2d(32, 64, 3, 1),
                              nn.ReLU(),
                              nn.MaxPool2d(2),
                              nn.Dropout(0.25),
                              nn.Flatten(),
                              nn.Linear(9216, 128),
                              nn.ReLU(),
                              nn.Dropout(0.5),
                              nn.Linear(128, 10),
                              nn.LogSoftmax(dim=1))
        return model
    
    def give_optimizer(self,lr):
        optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)
        return optimizer
    
    def train_model(self):
        self.model.train()

    def forward(self, x):
        return self.model(x)

    def training_data(self, batch_size = 48):
        # Custom torch Dataloader for MNIST data
        transform = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))])
        dataset1 = datasets.MNIST(self.dataset_path, train=True, download=False, transform=transform)
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        data_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
        return data_loader
    
    def training_step(self, data, target):
        output = self.forward(data)
        loss   = torch.nn.functional.nll_loss(output, target)
        return loss

Writing /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/tmp/tmpzjdnv4bu/class_export_mnist.py


This group of arguments correspond respectively:
* `model_args`: a dictionary with the arguments related to the model (e.g. number of layers, features, etc.). This will be passed to the model class on the node side. For instance, the privacy parameters should be passed here.
* `training_args`: a dictionary containing the arguments for the training routine (e.g. batch size, learning rate, epochs, etc.). This will be passed to the routine on the node side.

**NOTE:** typos and/or lack of positional (required) arguments will raise error. 🤓

In [5]:
model_args = {'diff_privacy': True, 'privacy_func': 'make_private', 'noise_multiplier':1., 'max_grad_norm':1.0}

training_args = {
    'batch_size': 48, 
    'lr': 1e-3, 
    'epochs': 3, 
    'dry_run': False,  
    'batch_maxnum': 250 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

## Declare and run the experiment

- search nodes serving data for these `tags`, optionally filter on a list of node ID with `nodes`
- run a round of local training on nodes with model defined in `model_path` + federation with `aggregator`
- run for `rounds` rounds, applying the `node_selection_strategy` between the rounds

In [6]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['#MNIST', '#dataset']
rounds = 3

exp = Experiment(tags=tags,
                 #nodes=None,
                 model_path=model_file,
                 model_args=model_args,
                 model_class='MyTrainingPlan',
                 training_args=training_args,
                 rounds=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None)

2022-01-20 13:02:56,828 fedbiomed INFO - Searching dataset with data tags: ['#MNIST', '#dataset'] for all nodes
2022-01-20 13:02:56,841 fedbiomed INFO - log from: node_063a33bf-26a8-436a-876e-ac5a0827c4b2 / DEBUG - Message received: {'researcher_id': 'researcher_1eef641b-73f6-45ab-b859-e37b8d87bbfd', 'tags': ['#MNIST', '#dataset'], 'command': 'search'}
2022-01-20 13:02:56,843 fedbiomed INFO - log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / DEBUG - Message received: {'researcher_id': 'researcher_1eef641b-73f6-45ab-b859-e37b8d87bbfd', 'tags': ['#MNIST', '#dataset'], 'command': 'search'}
2022-01-20 13:03:06,849 fedbiomed INFO - Node selected for training -> node_063a33bf-26a8-436a-876e-ac5a0827c4b2
2022-01-20 13:03:06,850 fedbiomed INFO - Node selected for training -> node_963c86ad-835e-424d-9bf5-73d8ec418720
2022-01-20 13:03:06,853 fedbiomed INFO - Checking data quality of federated datasets...
2022-01-20 13:03:07,384 fedbiomed DEBUG - torchnn saved model filename: /Users/balelli/o

Let's start the experiment.

By default, this function doesn't stop until all the `rounds` are done for all the nodes

In [7]:
exp.run()

2022-01-20 13:03:11,576 fedbiomed INFO - Sampled nodes in round 0 ['node_063a33bf-26a8-436a-876e-ac5a0827c4b2', 'node_963c86ad-835e-424d-9bf5-73d8ec418720']
01/20/2022 13:03:11:INFO:Sampled nodes in round 0 ['node_063a33bf-26a8-436a-876e-ac5a0827c4b2', 'node_963c86ad-835e-424d-9bf5-73d8ec418720']
2022-01-20 13:03:11,580 fedbiomed INFO - Send message to node node_063a33bf-26a8-436a-876e-ac5a0827c4b2 - {'researcher_id': 'researcher_1eef641b-73f6-45ab-b859-e37b8d87bbfd', 'job_id': '6a114b56-38ab-4f51-b934-7d17be95d92b', 'training_args': {'batch_size': 48, 'lr': 0.001, 'epochs': 3, 'dry_run': False, 'batch_maxnum': 250}, 'model_args': {'diff_privacy': True, 'privacy_func': 'make_private', 'noise_multiplier': 1.0, 'max_grad_norm': 1.0}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/01/20/my_model_5a925fe7-94eb-48f5-84cc-f73c7e71cdbe.py', 'params_url': 'http://localhost:8844/media/uploads/2022/01/20/aggregated_params_init_7e1b4306-069b-4ae9-853e-68cabdb55152.pt',

01/20/2022 13:03:11:INFO:log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / DEBUG - Message received: {'researcher_id': 'researcher_1eef641b-73f6-45ab-b859-e37b8d87bbfd', 'job_id': '6a114b56-38ab-4f51-b934-7d17be95d92b', 'training_args': {'batch_size': 48, 'lr': 0.001, 'epochs': 3, 'dry_run': False, 'batch_maxnum': 250}, 'model_args': {'diff_privacy': True, 'privacy_func': 'make_private', 'noise_multiplier': 1.0, 'max_grad_norm': 1.0}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/01/20/my_model_5a925fe7-94eb-48f5-84cc-f73c7e71cdbe.py', 'params_url': 'http://localhost:8844/media/uploads/2022/01/20/aggregated_params_init_7e1b4306-069b-4ae9-853e-68cabdb55152.pt', 'model_class': 'MyTrainingPlan', 'training_data': {'node_963c86ad-835e-424d-9bf5-73d8ec418720': ['dataset_8abb7dd7-2fbd-4c44-9cd7-3baf9fa82271']}}
2022-01-20 13:03:11,672 fedbiomed INFO - log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / DEBUG - [TASKS QUEUE] Item:{'researcher_id': 'researcher

01/20/2022 13:04:04:INFO:log from: node_063a33bf-26a8-436a-876e-ac5a0827c4b2 / INFO - Train Epoch: 1 [3220/60000 (6%)]	Loss: 2.194538
2022-01-20 13:04:05,293 fedbiomed INFO - log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / INFO - Train Epoch: 1 [3220/60000 (6%)]	Loss: 2.108488
01/20/2022 13:04:05:INFO:log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / INFO - Train Epoch: 1 [3220/60000 (6%)]	Loss: 2.108488
2022-01-20 13:04:10,695 fedbiomed INFO - log from: node_063a33bf-26a8-436a-876e-ac5a0827c4b2 / INFO - Train Epoch: 1 [4480/60000 (6%)]	Loss: 2.156969
01/20/2022 13:04:10:INFO:log from: node_063a33bf-26a8-436a-876e-ac5a0827c4b2 / INFO - Train Epoch: 1 [4480/60000 (6%)]	Loss: 2.156969
2022-01-20 13:04:11,695 fedbiomed INFO - log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / INFO - Train Epoch: 1 [3760/60000 (6%)]	Loss: 2.144862
01/20/2022 13:04:11:INFO:log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / INFO - Train Epoch: 1 [3760/60000 (6%)]	Loss: 2.144862
2022-01-20 13:

01/20/2022 13:05:31:INFO:log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / INFO - Train Epoch: 1 [10290/60000 (17%)]	Loss: 1.575786
2022-01-20 13:05:37,210 fedbiomed INFO - log from: node_063a33bf-26a8-436a-876e-ac5a0827c4b2 / INFO - Train Epoch: 1 [9900/60000 (18%)]	Loss: 1.599372
01/20/2022 13:05:37:INFO:log from: node_063a33bf-26a8-436a-876e-ac5a0827c4b2 / INFO - Train Epoch: 1 [9900/60000 (18%)]	Loss: 1.599372
2022-01-20 13:05:38,114 fedbiomed INFO - log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / INFO - Train Epoch: 1 [9900/60000 (18%)]	Loss: 1.687456
01/20/2022 13:05:38:INFO:log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / INFO - Train Epoch: 1 [9900/60000 (18%)]	Loss: 1.687456
2022-01-20 13:05:43,084 fedbiomed INFO - log from: node_063a33bf-26a8-436a-876e-ac5a0827c4b2 / INFO - Train Epoch: 1 [10810/60000 (18%)]	Loss: 1.460855
01/20/2022 13:05:43:INFO:log from: node_063a33bf-26a8-436a-876e-ac5a0827c4b2 / INFO - Train Epoch: 1 [10810/60000 (18%)]	Loss: 1.460855
2022

01/20/2022 13:06:56:INFO:log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / INFO - Train Epoch: 2 [4200/60000 (8%)]	Loss: 1.556576
2022-01-20 13:06:56,911 fedbiomed INFO - log from: node_063a33bf-26a8-436a-876e-ac5a0827c4b2 / INFO - Train Epoch: 2 [4400/60000 (8%)]	Loss: 1.438764
01/20/2022 13:06:56:INFO:log from: node_063a33bf-26a8-436a-876e-ac5a0827c4b2 / INFO - Train Epoch: 2 [4400/60000 (8%)]	Loss: 1.438764
2022-01-20 13:07:02,350 fedbiomed INFO - log from: node_063a33bf-26a8-436a-876e-ac5a0827c4b2 / INFO - Train Epoch: 2 [3960/60000 (9%)]	Loss: 1.320531
01/20/2022 13:07:02:INFO:log from: node_063a33bf-26a8-436a-876e-ac5a0827c4b2 / INFO - Train Epoch: 2 [3960/60000 (9%)]	Loss: 1.320531
2022-01-20 13:07:02,997 fedbiomed INFO - log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / INFO - Train Epoch: 2 [4180/60000 (9%)]	Loss: 1.415859
01/20/2022 13:07:02:INFO:log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / INFO - Train Epoch: 2 [4180/60000 (9%)]	Loss: 1.415859
2022-01-20 13:

...

01/20/2022 13:23:57:INFO:log from: node_063a33bf-26a8-436a-876e-ac5a0827c4b2 / INFO - Train Epoch: 1 [7920/60000 (14%)]	Loss: 0.915688
2022-01-20 13:24:02,536 fedbiomed INFO - log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / INFO - Train Epoch: 1 [9880/60000 (15%)]	Loss: 1.395708
01/20/2022 13:24:02:INFO:log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / INFO - Train Epoch: 1 [9880/60000 (15%)]	Loss: 1.395708
2022-01-20 13:24:03,383 fedbiomed INFO - log from: node_063a33bf-26a8-436a-876e-ac5a0827c4b2 / INFO - Train Epoch: 1 [10070/60000 (15%)]	Loss: 1.292268
01/20/2022 13:24:03:INFO:log from: node_063a33bf-26a8-436a-876e-ac5a0827c4b2 / INFO - Train Epoch: 1 [10070/60000 (15%)]	Loss: 1.292268
2022-01-20 13:24:08,161 fedbiomed INFO - log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / INFO - Train Epoch: 1 [8800/60000 (16%)]	Loss: 1.471374
01/20/2022 13:24:08:INFO:log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / INFO - Train Epoch: 1 [8800/60000 (16%)]	Loss: 1.471374
2022-

01/20/2022 13:25:23:INFO:log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / INFO - Train Epoch: 2 [3010/60000 (6%)]	Loss: 1.343298
2022-01-20 13:25:25,272 fedbiomed INFO - log from: node_063a33bf-26a8-436a-876e-ac5a0827c4b2 / INFO - Train Epoch: 2 [3290/60000 (6%)]	Loss: 0.781766
01/20/2022 13:25:25:INFO:log from: node_063a33bf-26a8-436a-876e-ac5a0827c4b2 / INFO - Train Epoch: 2 [3290/60000 (6%)]	Loss: 0.781766
2022-01-20 13:25:29,746 fedbiomed INFO - log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / INFO - Train Epoch: 2 [4000/60000 (6%)]	Loss: 1.509039
01/20/2022 13:25:29:INFO:log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / INFO - Train Epoch: 2 [4000/60000 (6%)]	Loss: 1.509039
2022-01-20 13:25:31,436 fedbiomed INFO - log from: node_063a33bf-26a8-436a-876e-ac5a0827c4b2 / INFO - Train Epoch: 2 [3360/60000 (6%)]	Loss: 1.060234
01/20/2022 13:25:31:INFO:log from: node_063a33bf-26a8-436a-876e-ac5a0827c4b2 / INFO - Train Epoch: 2 [3360/60000 (6%)]	Loss: 1.060234
2022-01-20 13:

01/20/2022 13:26:51:INFO:log from: node_063a33bf-26a8-436a-876e-ac5a0827c4b2 / INFO - Train Epoch: 2 [8400/60000 (17%)]	Loss: 1.250928
2022-01-20 13:26:56,160 fedbiomed INFO - log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / INFO - Train Epoch: 2 [9900/60000 (18%)]	Loss: 1.826025
01/20/2022 13:26:56:INFO:log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / INFO - Train Epoch: 2 [9900/60000 (18%)]	Loss: 1.826025
2022-01-20 13:26:57,196 fedbiomed INFO - log from: node_063a33bf-26a8-436a-876e-ac5a0827c4b2 / INFO - Train Epoch: 2 [11660/60000 (18%)]	Loss: 0.900939
01/20/2022 13:26:57:INFO:log from: node_063a33bf-26a8-436a-876e-ac5a0827c4b2 / INFO - Train Epoch: 2 [11660/60000 (18%)]	Loss: 0.900939
2022-01-20 13:27:02,012 fedbiomed INFO - log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / INFO - Train Epoch: 2 [12190/60000 (18%)]	Loss: 0.662904
01/20/2022 13:27:02:INFO:log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / INFO - Train Epoch: 2 [12190/60000 (18%)]	Loss: 0.662904
202

01/20/2022 13:28:17:INFO:log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / INFO - Train Epoch: 3 [4400/60000 (8%)]	Loss: 0.561417
2022-01-20 13:28:20,527 fedbiomed INFO - log from: node_063a33bf-26a8-436a-876e-ac5a0827c4b2 / INFO - Train Epoch: 3 [5700/60000 (8%)]	Loss: 1.243619
01/20/2022 13:28:20:INFO:log from: node_063a33bf-26a8-436a-876e-ac5a0827c4b2 / INFO - Train Epoch: 3 [5700/60000 (8%)]	Loss: 1.243619
2022-01-20 13:28:25,218 fedbiomed INFO - log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / INFO - Train Epoch: 3 [5280/60000 (9%)]	Loss: 1.341679
01/20/2022 13:28:25:INFO:log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / INFO - Train Epoch: 3 [5280/60000 (9%)]	Loss: 1.341679
2022-01-20 13:28:28,025 fedbiomed INFO - log from: node_063a33bf-26a8-436a-876e-ac5a0827c4b2 / INFO - Train Epoch: 3 [4730/60000 (9%)]	Loss: 1.599997
01/20/2022 13:28:28:INFO:log from: node_063a33bf-26a8-436a-876e-ac5a0827c4b2 / INFO - Train Epoch: 3 [4730/60000 (9%)]	Loss: 1.599997
2022-01-20 13:

01/20/2022 13:29:46:INFO:log from: node_063a33bf-26a8-436a-876e-ac5a0827c4b2 / INFO - Train Epoch: 3 [13920/60000 (19%)]	Loss: 1.230220
2022-01-20 13:29:48,982 fedbiomed INFO - log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / DEBUG - Reached 250 batches for this epoch, ignore remaining data
01/20/2022 13:29:48:INFO:log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / DEBUG - Reached 250 batches for this epoch, ignore remaining data
2022-01-20 13:29:50,060 fedbiomed INFO - log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / INFO - results uploaded successfully 
01/20/2022 13:29:50:INFO:log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / INFO - results uploaded successfully 
2022-01-20 13:29:52,801 fedbiomed INFO - log from: node_063a33bf-26a8-436a-876e-ac5a0827c4b2 / DEBUG - Reached 250 batches for this epoch, ignore remaining data
01/20/2022 13:29:52:INFO:log from: node_063a33bf-26a8-436a-876e-ac5a0827c4b2 / DEBUG - Reached 250 batches for this epoch, ignore remaining data
20

Local training results for each round and each node are available in `exp.training_replies` (index 0 to (`rounds` - 1) ).

For example you can view the training results for the last round below.

Different timings (in seconds) are reported for each dataset of a node participating in a round :
- `rtime_training` real time (clock time) spent in the training function on the node
- `ptime_training` process time (user and system CPU) spent in the training function on the node
- `rtime_total` real time (clock time) spent in the researcher between sending the request and handling the response, at the `Job()` layer

In [8]:
print("\nList the training rounds : ", exp.training_replies.keys())

print("\nList the nodes for the last training round and their timings : ")
round_data = exp.training_replies[rounds - 1].data
for c in range(len(round_data)):
    print("\t- {id} :\
    \n\t\trtime_training={rtraining:.2f} seconds\
    \n\t\tptime_training={ptraining:.2f} seconds\
    \n\t\trtime_total={rtotal:.2f} seconds".format(id = round_data[c]['node_id'],
        rtraining = round_data[c]['timing']['rtime_training'],
        ptraining = round_data[c]['timing']['ptime_training'],
        rtotal = round_data[c]['timing']['rtime_total']))
print('\n')
    
exp.training_replies[rounds - 1].dataframe


List the training rounds :  dict_keys([0, 1, 2])

List the nodes for the last training round and their timings : 
	- node_963c86ad-835e-424d-9bf5-73d8ec418720 :    
		rtime_training=463.64 seconds    
		ptime_training=651.12 seconds    
		rtime_total=475.37 seconds
	- node_063a33bf-26a8-436a-876e-ac5a0827c4b2 :    
		rtime_training=467.50 seconds    
		ptime_training=655.58 seconds    
		rtime_total=475.65 seconds




,success,msg,dataset_id,node_id,params_path,params,timing
0,True,,dataset_8abb7dd7-2fbd-4c44-9cd7-3baf9fa82271,node_963c86ad-835e-424d-9bf5-73d8ec418720,/Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed...,"{'model._module.0.weight': [[tensor([[-0.2284,...","{'rtime_training': 463.6361137949998, 'ptime_t..."
1,True,,dataset_1ed69726-f63a-4176-84b7-1af3708adb38,node_063a33bf-26a8-436a-876e-ac5a0827c4b2,/Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed...,"{'model._module.0.weight': [[tensor([[-0.1184,...","{'rtime_training': 467.5021604560002, 'ptime_t..."


Federated parameters for each round are available in `exp.aggregated_params` (index 0 to (`rounds` - 1) ).

For example you can view the federated parameters for the last round of the experiment :

In [9]:
print("\nList the training rounds : ", exp.aggregated_params.keys())

print("\nAccess the federated params for the last training round :")
print("\t- params_path: ", exp.aggregated_params[rounds - 1]['params_path'])
print("\t- parameter data: ", exp.aggregated_params[rounds - 1]['params'].keys())



List the training rounds :  dict_keys([0, 1, 2])

Access the federated params for the last training round :
	- params_path:  /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/experiments/Experiment_0003/aggregated_params_0f454c44-de07-4a85-baa2-44196079163a.pt
	- parameter data:  odict_keys(['model.0.weight', 'model.0.bias', 'model.2.weight', 'model.2.bias', 'model.7.weight', 'model.7.bias', 'model.10.weight', 'model.10.bias'])


# Testing

We define a little testing routine to extract the accuracy metrics on the testing dataset

In [10]:
import torch
import torch.nn.functional as F


def testing_Accuracy(model, data_loader):
    model.eval()
    test_loss = 0
    correct = 0
    device = 'cpu'

    correct = 0
    
    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

        pred = output.argmax(dim=1, keepdim=True)

    test_loss /= len(data_loader.dataset)
    accuracy = 100* correct/len(data_loader.dataset)

    return(test_loss, accuracy)

In [11]:
from torchvision import datasets, transforms
import os

local_mnist = os.path.join(environ['TMP_DIR'], 'local_mnist')

transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])

test_set = datasets.MNIST(root = local_mnist, download = True, train = False, transform = transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=True)

fed_model = exp.model_instance
fed_model.load_state_dict(exp.aggregated_params[rounds - 1]['params'])

acc_federated = testing_Accuracy(fed_model, test_loader)

print('\nAccuracy federated training:  {:.4f}'.format(acc_federated[1]))

print('\nError federated training:  {:.4f}'.format(acc_federated[0]))

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/tmp/local_mnist/MNIST/raw/train-images-idx3-ubyte.gz to /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/tmp/local_mnist/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/tmp/local_mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/tmp/local_mnist/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

...

Done!

Accuracy federated training:  86.4200

Error federated training:  0.5659


2022-01-20 13:32:43,706 fedbiomed INFO - log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / DEBUG - Message received: {'researcher_id': 'researcher_1eef641b-73f6-45ab-b859-e37b8d87bbfd', 'tags': ['#MNIST', '#dataset'], 'command': 'search'}
01/20/2022 13:32:43:INFO:log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / DEBUG - Message received: {'researcher_id': 'researcher_1eef641b-73f6-45ab-b859-e37b8d87bbfd', 'tags': ['#MNIST', '#dataset'], 'command': 'search'}
2022-01-20 13:32:43,735 fedbiomed INFO - log from: node_063a33bf-26a8-436a-876e-ac5a0827c4b2 / DEBUG - Message received: {'researcher_id': 'researcher_1eef641b-73f6-45ab-b859-e37b8d87bbfd', 'tags': ['#MNIST', '#dataset'], 'command': 'search'}
01/20/2022 13:32:43:INFO:log from: node_063a33bf-26a8-436a-876e-ac5a0827c4b2 / DEBUG - Message received: {'researcher_id': 'researcher_1eef641b-73f6-45ab-b859-e37b8d87bbfd', 'tags': ['#MNIST', '#dataset'], 'command': 'search'}
2022-01-20 13:32:58,691 fedbiomed INFO - log from: node_963c

2022-01-20 13:32:59,809 fedbiomed INFO - log from: node_063a33bf-26a8-436a-876e-ac5a0827c4b2 / DEBUG - Dataset_path/Users/balelli/data
01/20/2022 13:32:59:INFO:log from: node_063a33bf-26a8-436a-876e-ac5a0827c4b2 / DEBUG - Dataset_path/Users/balelli/data
2022-01-20 13:33:00,244 fedbiomed INFO - log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / INFO - Train Epoch: 1 [0/60000 (0%)]	Loss: 2.296201
01/20/2022 13:33:00:INFO:log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / INFO - Train Epoch: 1 [0/60000 (0%)]	Loss: 2.296201
2022-01-20 13:33:00,305 fedbiomed INFO - log from: node_063a33bf-26a8-436a-876e-ac5a0827c4b2 / INFO - Train Epoch: 1 [0/60000 (0%)]	Loss: 2.297313
01/20/2022 13:33:00:INFO:log from: node_063a33bf-26a8-436a-876e-ac5a0827c4b2 / INFO - Train Epoch: 1 [0/60000 (0%)]	Loss: 2.297313
2022-01-20 13:33:02,994 fedbiomed INFO - log from: node_063a33bf-26a8-436a-876e-ac5a0827c4b2 / INFO - Train Epoch: 1 [480/60000 (1%)]	Loss: 1.256780
01/20/2022 13:33:02:INFO:log from: node_0

2022-01-20 13:33:25,092 fedbiomed INFO - log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / DEBUG - [TASKS QUEUE] Item:{'researcher_id': 'researcher_1eef641b-73f6-45ab-b859-e37b8d87bbfd', 'job_id': '1e328907-cff6-451a-a053-c7c64d23c189', 'params_url': 'http://localhost:8844/media/uploads/2022/01/20/aggregated_params_3449ef32-b812-430b-ad4a-ef8421cf223f.pt', 'training_args': {'batch_size': 48, 'lr': 0.001, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 100}, 'training_data': {'node_963c86ad-835e-424d-9bf5-73d8ec418720': ['dataset_8abb7dd7-2fbd-4c44-9cd7-3baf9fa82271']}, 'model_args': {}, 'model_url': 'http://localhost:8844/media/uploads/2022/01/20/my_model_d43bb215-0f59-46f6-a7cd-3ed0ff7bab8f.py', 'model_class': 'MyTrainingPlan', 'command': 'train'}
01/20/2022 13:33:25:INFO:log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / DEBUG - [TASKS QUEUE] Item:{'researcher_id': 'researcher_1eef641b-73f6-45ab-b859-e37b8d87bbfd', 'job_id': '1e328907-cff6-451a-a053-c7c64d23c189', 'params_url':

2022-01-20 13:33:32,538 fedbiomed INFO - log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / INFO - Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.154684
01/20/2022 13:33:32:INFO:log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / INFO - Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.154684
2022-01-20 13:33:32,643 fedbiomed INFO - log from: node_063a33bf-26a8-436a-876e-ac5a0827c4b2 / INFO - Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.187952
01/20/2022 13:33:32:INFO:log from: node_063a33bf-26a8-436a-876e-ac5a0827c4b2 / INFO - Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.187952
2022-01-20 13:33:33,909 fedbiomed INFO - log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / INFO - Train Epoch: 1 [2400/60000 (4%)]	Loss: 0.220249
01/20/2022 13:33:33:INFO:log from: node_963c86ad-835e-424d-9bf5-73d8ec418720 / INFO - Train Epoch: 1 [2400/60000 (4%)]	Loss: 0.220249
2022-01-20 13:33:34,050 fedbiomed INFO - log from: node_063a33bf-26a8-436a-876e-ac5a0827c4b2 / INFO - Train Epoch: 1 [2400/60000 (4%)]	Loss: 0.14477